In [1]:
import os
import numpy as np
import roboticstoolbox as rtb
from roboticstoolbox.robot.ERobot import ERobot
import matplotlib as plt
import matplotlib . pyplot as plt
import mpl_toolkits . mplot3d . art3d as art3d
from matplotlib . patches import Rectangle
from spatialgeometry import Cuboid
import spatialmath.base as smb
from spatialmath import SE3
from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')
from mpl_toolkits.mplot3d import axes3d   
from matplotlib.colors import ListedColormap
from bresenham import bresenham
import random
from tqdm import tqdm
import imageio

In [2]:
class TwoLink(ERobot):
    '''
    Initialize a 2link robot
    '''
    def __init__(self):
        links, name, urdf_string, urdf_filepath = self.URDF_read("two_link.urdf", tld=os.getcwd())
        super().__init__(
            links,
            name=name,
            manufacturer="ARClab",
            urdf_string=urdf_string,
            urdf_filepath=urdf_filepath,
        )
        
        self.qr = np.array([0, 0.0])
        self.qz = np.zeros(2)

        self.addconfiguration("qr", self.qr)
        self.addconfiguration("qz", self.qz)

In [3]:
# Creating rectange patch function
def add_rectangle ( xy , width , height , ax , col) :
# ’’’
# Add a rectangle patch to the environment .
# param xy: xy corner of the left - bottom corner .
# param height : height of the window .
# param width : width of the window .
# param ax: the axis on which to plot .
# ’’’
    obj_patch = Rectangle ( xy , height , width, color = col )
    ax . add_patch ( obj_patch )
    art3d . pathpatch_2d_to_3d ( obj_patch , z = 0 , zdir ='z')

In [4]:
two_link = TwoLink()

fig = plt . figure ()

# Plot the robot pose
plot_fig = two_link . plot ( q =[0 ,0] , fig = fig, backend ='pyplot' )

# Setting environment limits
plot_fig.ax.set_xlim(-40,40)
plot_fig.ax.set_ylim(-35,35)

# Obstacle 1 top left
add_rectangle ((-12 , 5 ) , 30 , 1 , plot_fig.ax, 'green')
rect1 = Cuboid ( scale =[ 1, 30, 1 ] , pose = SE3(-11.5, 20, 0) , collision = True )

# Obstacle 2 bottom left
add_rectangle ((-12 , -35 ) , 35 , 1 , plot_fig.ax, 'blue')
rect2 = Cuboid ( scale =[ 1, 35, 1 ] , pose = SE3(-11.5, -17.5, 0) , collision = True )

# Obstacle 3 bottom right
add_rectangle ((11 , -35 ) , 30 , 1 , plot_fig.ax, 'yellow')
rect3 = Cuboid ( scale =[ 1, 30, 1 ] , pose = SE3(11.5, -20, 0) , collision = True )

# Obstacle 4 top right
add_rectangle ((11 , 0 ) , 35 , 1 , plot_fig.ax, 'purple')
rect4 = Cuboid ( scale =[ 1, 35, 1 ] , pose = SE3(11.5, 17.5, 0) , collision = True )

# set plot view to the X-Y axis .
plot_fig.ax.view_init (90 , -90 , 0)
plt.show()
%qtconsole

![alt text](Environment.png "Title")

In [5]:
# Collision checking and constructing configuration space
conf_mat = np.zeros((350,360))
for i in range(360):
    for j in range(-175,175):
        ang1 = i*np.pi/180
        ang2 = j*np.pi/180
        if (two_link.iscollided([ang1,ang2], rect1)):
            j = j+175
            conf_mat[j,i] = 0.25
        elif (two_link.iscollided([ang1,ang2], rect2)): 
            j = j+175
            conf_mat[j,i] = 0.5
        elif (two_link.iscollided([ang1,ang2], rect3)): 
            j = j+175
            conf_mat[j,i] = 0.75
        elif (two_link.iscollided([ang1,ang2], rect4)): 
            j = j+175
            conf_mat[j,i] = 1




In [6]:
cmap = ListedColormap(['white','g','b','yellow','purple'])
plt.imshow(conf_mat, cmap = cmap, origin='lower', extent=[-180,180,-175,175])
plt.xlabel(r"${\Theta}_1$[degrees]",fontsize = 20)
plt.ylabel(r"${\Theta}_2$[degrees]",fontsize = 20)
plt.title('Configuration Space with collision free (white space) and collision regions (corresponding colours from environment plot)',fontsize = 20)
plt.show()

![alt text](Conf_space.png "Title")

## RRT* Algorithm

In [7]:
# Waypoints to track using the RRT* algorithm: All z coordinates are added as zero.
wayp = [[0, 20, 0], ## Start point for the robot
        [0, -20, 0],
        [-19.43, 4.62, 0],
        [2, 2, 0],
        [-8, 0, 0],
        [-5, -5, 0],
        [-9, 0, 0]]

# Finding corresponding angles to track in configuration space:
angles_way = []
prev_wayp = list(two_link.ikine_LM(SE3(wayp[0]))[0])

for waypoint in wayp:
    ang = list(two_link.ikine_LM(SE3(waypoint),prev_wayp)[0])
    angles_way.append(ang)
    prev_wayp = ang # seeding to find solution in each case

thetas = list(np.round((np.array(angles_way)*180/np.pi) + np.array([[180,175]])).astype(int))
for i in range(7):
    thetas[i] = list(thetas[i])
print(thetas)

conf_mat = conf_mat.T # Transpose of matrix to be easy for configuration matrix to run

[[273, 168], [80, 195], [341, 186], [311, 11], [66, 42], [114, 36], [63, 48]]


In [8]:
# Sample free function:
def sample_free(i,end_pos):
    x = random.randint(0, 360)
    y = random.randint(0, 350)
    return [x,y] # returns a randomly generated point from the available points

In [9]:
# Nearest function to find nearest node
def nearest(V, E, x_rand):
    temp = []
    temp = temp + V
    for tuple in E:
        first = tuple[0]
        second = tuple[1]
        if first not in temp:
            temp.append(first)
        if second not in temp:
            temp.append(second)
    idx = np.argmin(np.linalg.norm((np.array(temp) - np.array(x_rand)),axis=1))
    return temp[idx] # returns the nearest point to the x_rand among the points in V

In [10]:
# Steer function to steer and create a new node x_new 
def steer(x_nearest, x_rand):
    eps = 10
    direction = np.array(x_rand) - np.array(x_nearest)
    length = ((direction[0])**2 + (direction[1])**2)**0.5
    if length == 0:
        x_new = x_nearest
        return x_new
    unit_vector = [direction[0]/length, direction[1]/length]
    x_new = [np.round(x_nearest[0] + eps*unit_vector[0]).astype(int), np.round(x_nearest[1] + eps*unit_vector[1]).astype(int)]
    if x_new[0] > 359:
        x_new[0] = 359
    if x_new[1] > 349:
        x_new[1] = 349
    if x_new[0] < 0:
        x_new[0] = 0
    if x_new[1] < 0:
        x_new[1] = 0
    return x_new # returns new point along the x_rand direction from x_nearest along epsilon distance in that direction

In [11]:
# Function to check collision between nearest and the new node
def collision_free(x_nearest, x_new, map):
    bres = np.array(list(bresenham(x_nearest[0], x_nearest[1], x_new[0], x_new[1])))
    for coord in bres:
        if (map[coord[0],coord[1]] != 0):
            return False
    return True

In [12]:
# Near function to find near nodes in the radius r* around x_new
def near_fun(V, E, x_new):
    temp = []
    temp = temp + V
    for tuple in E:
        first = tuple[0]
        second = tuple[1]
        if first not in temp:
            temp.append(first)
        if second not in temp:
            temp.append(second)
    Xnear_list = []
    r_star = 20
    for node in temp:
        if (((node[0]-x_new[0])**2 + (node[1]-x_new[1])**2)**0.5 <= r_star):
            Xnear_list.append(node)
    return Xnear_list # returns list of near nodes in the vicinity

In [13]:
def RRTstar_planner(map, start_pos, end_pos):
    V = [start_pos]
    E = []
    cost = np.ones((360,350))*np.inf
    parent = [[[] for j in range(350)] for i in range(360)]
    tups = (start_pos[0], start_pos[1])
    cost[start_pos[0],start_pos[1]] = 0
    flag = 0
    for i in tqdm(range(2000)):
        x_rand = sample_free(i, end_pos)
        x_nearest = nearest(V,E,x_rand)
        x_new = steer(x_nearest, x_rand)
        if collision_free(x_nearest, x_new, map):
            Xnear_list = near_fun(V,E,x_new)
            x_min = x_nearest
            cmin = cost[x_nearest[0],x_nearest[1]] + np.linalg.norm(np.asarray(x_nearest)-np.asarray(x_new))
            cost[x_new[0],x_new[1]] = cmin

            ## First tree construction:
            for x_near in Xnear_list:
                if collision_free(x_near,x_new,map):
                    c = cost[x_near[0],x_near[1]] + np.linalg.norm(np.asarray(x_near)-np.asarray(x_new))
                    if c < cmin:
                        x_min = x_near
                        cmin = c
            
            V.append(x_new)
            E.append((x_min,x_new))

            if x_new != x_min:
                parent[x_new[0]][x_new[1]] = np.array(x_min)      
            # cmin = cost[x_nearest[0],x_nearest[1]] + np.linalg.norm(np.asarray(x_nearest)-np.asarray(x_new))
            
            ## Rewire step:
            for x_near in Xnear_list:
                if x_near != x_min:
                    if collision_free(x_near,x_new,map):
                        c_ = cost[x_new[0],x_new[1]] + np.linalg.norm(np.asarray(x_near)-np.asarray(x_new))
                        if (c_ < cost[x_near[0],x_near[1]]):
                            if x_near != x_new:
                                parent[x_near[0]][x_near[1]] = np.array(x_new)
                            # x_min = x_near
                            E.append((x_min,x_new))
        if flag == 0:
            goal_node1 = nearest(V,E,end_pos)
            if collision_free(goal_node1,end_pos,map):
                parent1 = parent.copy()
                flag = 1
                
    goal_node = nearest(V,E,end_pos)
    return parent1, goal_node1, V, E, parent, goal_node


In [14]:
def gif_creator(map, start_pos, end_pos, parents1, goal_node1, V, E, parents, goal_node): ## Constructing the tree with vertices and edges
    # Create a figure and an axis object
    fig, ax = plt.subplots()
    
    # Initialize an empty line object
    line, = ax.plot([], [], 'o-')

    for tuple in E:
        first = tuple[0]
        second = tuple[1]
        plt.plot([first[0]-180, second[0]-180],[first[1]-175, second[1]-175], 'k-', linewidth=0.5)

    cmap = ListedColormap(['white','g','b','yellow','purple'])
    plt.imshow(conf_mat.T, cmap = cmap,origin='lower', extent=[-180,180,-175,175])
    V_array = np.array(V)
    plt.scatter(V_array[:,0]-180,V_array[:,1]-175, s= 10, color = 'yellow')


    # Find first path
    opt_path1 = []
    target = end_pos
    opt_path1.append(target)
    current_node = goal_node1
    start_node = start_pos

    while np.all(current_node != start_node):
        opt_path1.append(current_node)
        current_node = parents1[current_node[0]][current_node[1]]


    opt_path1.append(np.array(start_node))
    opt_path1.reverse()
    opt_path1 = np.array(opt_path1)


    # Find optimum path
    opt_path = []
    target = end_pos
    opt_path.append(target)
    current_node = goal_node
    start_node = start_pos

    while np.all(current_node != start_node):
        opt_path.append(current_node)
        current_node = parents[current_node[0]][current_node[1]]

    opt_path.append(np.array(start_node))
    opt_path.reverse()
    opt_path = np.array(opt_path)

    plt.plot(opt_path1[:,0]-180,opt_path1[:,1]-175,'b',linewidth = 2)
    # plt.plot(opt_path[:,0]-180,opt_path[:,1]-175,'r',linewidth = 3)
    
    # Show start and goal points
    plt.scatter(start_pos[0]-180, start_pos[1]-175, color = 'purple', s = 64)
    plt.scatter(end_pos[0]-180, end_pos[1]-175, color = 'g', s = 64)

    # Animation update function
    def update(frame):
        # Select points up to the current frame
        current_points = opt_path[:frame+1]

        # Update the line data
        line.set_data(current_points[:, 0]-180, current_points[:, 1]-175)
        
        line.set_color('r')
        line.set_linewidth(3)

        return line,

    # Create the animation
    animation = mpl.animation.FuncAnimation(fig, update, frames=len(opt_path), interval=100, blit=True)

    # Save the animation as a GIF
    animation.save('animation.gif', writer='pillow')
    
    angles = (opt_path - np.array([[180,175]]))*np.pi/180

    plt.show()
    
    return angles

In [36]:
start_pos = [114,36]
end_pos = [63,48]
parents1, goal_node1, V, E, parents, goal_node = RRTstar_planner(conf_mat, start_pos, end_pos)

100%|██████████| 2000/2000 [00:50<00:00, 39.54it/s] 


In [37]:
angles = gif_creator(conf_mat,start_pos,end_pos, parents1, goal_node1, V, E, parents, goal_node)

In [39]:

two_link = TwoLink()
for i in range(angles.shape[0]):

    fig = plt . figure ()

    # Plot the robot pose
    plot_fig = two_link . plot ( angles[i,:] , backend ='pyplot' )

    # Setting environment limits
    plot_fig.ax.set_xlim(-40,40)
    plot_fig.ax.set_ylim(-35,35)

    # Obstacle 1 top left
    add_rectangle ((-12 , 5 ) , 30 , 1 , plot_fig.ax, 'green')
    rect1 = Cuboid ( scale =[ 1, 30, 1 ] , pose = SE3(-11.5, 20, 0) , collision = True )

    # Obstacle 2 bottom left
    add_rectangle ((-12 , -35 ) , 35 , 1 , plot_fig.ax, 'blue')
    rect2 = Cuboid ( scale =[ 1, 35, 1 ] , pose = SE3(-11.5, -17.5, 0) , collision = True )

    # Obstacle 3 bottom right
    add_rectangle ((11 , -35 ) , 30 , 1 , plot_fig.ax, 'yellow')
    rect3 = Cuboid ( scale =[ 1, 30, 1 ] , pose = SE3(11.5, -20, 0) , collision = True )

    # Obstacle 4 top right
    add_rectangle ((11 , 0 ) , 35 , 1 , plot_fig.ax, 'purple')
    rect4 = Cuboid ( scale =[ 1, 35, 1 ] , pose = SE3(11.5, 17.5, 0) , collision = True )
    
    plt.scatter(-5, -5, color = 'red', s = 64)
    plt.scatter(-9, 0, color = 'green', s = 64)

    # set plot view to the X-Y axis .
    plot_fig.ax.view_init (90 , -90 , 0)
    plt.savefig(f"{i}.png")
    plt.close(fig)

